In [ ]:
!pip install roboflow opencv-python-headless matplotlib requests tqdm scikit-learn pandas numpy pillow

In [ ]:
import os
import cv2
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
from tqdm import tqdm
import requests
import io
from typing import Dict, List, Tuple, Optional
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score, f1_score,
    average_precision_score, precision_recall_curve
)
from roboflow import Roboflow

In [ ]:
CONFIG = {
    'roboflow_api_key': "K0xg5GEEinqPgaqjKKzz",
    'workspace': "matyworkspace",
    'project': "damagedhealthytrafficsigns",
    'version': 9,
    'analysis_property': 'contrast'
}
CONFIG_YOLO = {
    'roboflow_api_key': "LcrgBgwaWa5TXeG1Vtxa",
    'workspace': "matyworkspace",
    'project': "just-traffic-signs-45axx",
    'version': 3,
    'analysis_property': 'contrast'
}
CONFIG_EFNET = {
    'roboflow_api_key': "K0xg5GEEinqPgaqjKKzz",
    'workspace': "matyworkspace",
    'project': "traffic-signs-cl-damaged-train",
    'version': 1,
    'analysis_property': 'contrast'
}

In [ ]:
# Load dataset from Roboflow
rf = Roboflow(api_key=CONFIG['roboflow_api_key'])
project = rf.workspace(CONFIG['workspace']).project(CONFIG['project'])
version = project.version(CONFIG['version'])
dataset = version.download("yolov8")
#dataset = version.download("folder")

# Set up paths
image_dir = os.path.join(dataset.location, "test", "images")
original_image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))
#image_dir = os.path.join(dataset.location, "train", "not_ok")
#original_image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))
#image_dir = os.path.join(dataset.location, "train", "ok")
#original_image_paths = original_image_paths + glob.glob(os.path.join(image_dir, "*.jpg"))

print(f"Loaded {len(original_image_paths)} images from dataset")

In [ ]:
def measure_image_properties(image_path: str) -> Dict[str, float]:
    """
    Measure various properties of an image including brightness, noise, saturation, contrast, and blur.
    """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Measure brightness (mean luminance)
    brightness = np.mean(gray) / 255.0
    
    # Measure contrast (standard deviation of pixel intensities)
    contrast = np.std(gray) / 255.0
    
    # Measure saturation (mean saturation in HSV)
    saturation = np.mean(hsv[:, :, 1]) / 255.0
    
    # Measure noise using Laplacian variance
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    noise_variance = laplacian.var()
    noise_level = 1.0 / (1.0 + noise_variance / 1000.0)
    
    # Measure blur
    blur_measure = cv2.Laplacian(gray, cv2.CV_64F).var()
    normalized_blur = min(1.0, blur_measure / 1000.0)
    
    return {
        'brightness': float(brightness),
        'contrast': float(contrast),
        'saturation': float(saturation),
        'noise_level': float(noise_level),
        'blur_measure': float(normalized_blur)
    }

In [ ]:
# Measure the selected property for all images in the dataset
property_to_analyze = CONFIG['analysis_property']
print(f"Analyzing {property_to_analyze} for {len(original_image_paths)} images...")

property_values = []
image_names = []

for image_path in tqdm(original_image_paths, desc=f"Measuring {property_to_analyze}"):
    try:
        properties = measure_image_properties(image_path)
        property_values.append(properties[property_to_analyze])
        image_names.append(os.path.basename(image_path))
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        continue

print(f"Successfully analyzed {len(property_values)} images")

# Create histogram of the selected property
plt.figure(figsize=(12, 8))
plt.hist(property_values, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.title(f'Distribution of {property_to_analyze.capitalize()} Values Across Dataset', fontsize=16)
plt.xlabel(f'{property_to_analyze.capitalize()} Value', fontsize=12)
plt.ylabel('Number of Images', fontsize=12)
plt.grid(True, alpha=0.3)

# Add statistics to the plot
mean_val = np.mean(property_values)
std_val = np.std(property_values)
plt.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.3f}')
plt.axvline(mean_val + std_val, color='orange', linestyle='--', alpha=0.7, label=f'Mean + Std: {mean_val + std_val:.3f}')
plt.axvline(mean_val - std_val, color='orange', linestyle='--', alpha=0.7, label=f'Mean - Std: {mean_val - std_val:.3f}')
plt.legend()

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\n{property_to_analyze.capitalize()} Statistics:")
print(f"Mean: {mean_val:.4f}")
print(f"Std Dev: {std_val:.4f}")
print(f"Min: {min(property_values):.4f}")
print(f"Max: {max(property_values):.4f}")
print(f"Median: {np.median(property_values):.4f}")

# Create a DataFrame for easier analysis
df_analysis = pd.DataFrame({
    'image_name': image_names,
    property_to_analyze: property_values
})

print(f"\nDataFrame created with {len(df_analysis)} entries")
df_analysis.head()